In [1]:
!pip install wandb; git clone https://github.com/huggingface/transformers; cd transformers; pip install .

     |████████████████████████████████| 1.8MB 5.5MB/s 
     |████████████████████████████████| 163kB 18.6MB/s 
     |████████████████████████████████| 102kB 8.5MB/s 
     |████████████████████████████████| 102kB 8.3MB/s 
     |████████████████████████████████| 133kB 16.6MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.4-cp36-none-any.whl size=74841 sha256=5d94d0bea8a7c7b83819b485636ec473a12bdd822350fa96a592adddfe1a6667
  Stored in directory: /root/.cache/pip/wheels/9e/11/04/5160b8815b0cc7cf574bdc6d053e510169ec264c8791b4ec3a
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=bb1dcc8e04636f95c5eac0d2c15473d1e5ceab42d6b3b2b325d619a204f496a1
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=c99d36c6b75421827fd4a398038d4ac1cfb5cc6a5ceaf2

In [1]:
import os
import pickle
from tqdm import tqdm

import pandas as pd
import torch
import transformers
from tqdm import tqdm


In [2]:
tokenizer_path = 'drive/MyDrive/data/wine/gpt2_large/'
tokenizer = transformers.GPT2Tokenizer.from_pretrained(tokenizer_path)
print("Loaded tokenizer")

OSError: Can't load tokenizer for 'drive/MyDrive/data/wine/gpt2_large/'. Make sure that:

- 'drive/MyDrive/data/wine/gpt2_large/' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'drive/MyDrive/data/wine/gpt2_large/' is the correct path to a directory containing relevant tokenizer files



In [4]:
names_path = 'drive/MyDrive/data/wine/fake_names_12184_2020-11-19.pickle'
with open(names_path, 'rb',) as file:
    fake_wine_names = pickle.load(file)

In [5]:
fake_wine_names[:5]

['Piul Bothen Cabernet Sauvignon 2014',
 'Seacuscini Resantzass 2015',
 'Alpanena Vriestioge 2016',
 'Sanmedarbecer Edena Cabernet',
 'Sauvignon 2001']

In [6]:
model_path = 'drive/MyDrive/data/wine/checkpoints/step_95000_loss_0.7529/'
model = transformers.AutoModelForCausalLM.from_pretrained(model_path)
model = model.to('cuda')
print("Loaded model")

Loaded model


In [7]:
tokenizer.encode('<|endoftext|>')

[50256]

In [9]:
generated_descriptions = {}
for fake_name in tqdm(fake_wine_names):
  # Create token from fake wine name
  input_ids = tokenizer.encode(
    text=("<|startoftext|> [prompt] " + fake_name + " " + "[response] "),
    return_tensors='pt'
  ).to('cuda')

  # Generate a fake description based on the name
  model_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=250, 
    top_p=0.8,
    top_k=200,
    temperature=0.9,
    eos_token_id=50257,
    bos_token_id=50256,
    pad_token_id=50256,
    early_stopping=True
  )
  
  generated_descriptions[fake_name] = tokenizer.decode(
    token_ids=model_output[0], 
    skip_special_tokens=True
  )


  0%|          | 0/12184 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")

  3%|▎         | 423/12184 [1:13:51<33:06:48, 10.14s/it]

KeyboardInterrupt: ignored

In [11]:
for name in generated_descriptions.keys():
  print(generated_descriptions[name])
  break

[prompt] Piul Bothen Cabernet Sauvignon 2014 [response] [category_1] "Red Wine" [category_2] "Cabernet Sauvignon" [origin] " from Napa Valley, California" [description] "Deep garnet in color, this wine opens with rich aromas of blackberry, blueberry and cassis, complemented by a delicate touch of spice and vanilla. The palate is lush with flavors of dark fruit and baking spice. The finish is long and smooth, showcasing beautiful structure and layered tannins."     Wine News Top 100 of 2019" "Wine Spectator Top 100 of 2019" "This wine has the pedigree to age for over a decade. This wine is delicious with roasted pork tenderloin, and lamb chops." Wine Spectator Top 100 of 2019" "The 2014 vintage is an excellent wine, pairing with savory dishes, aged cheeses, and rich pasta dishes. Enjoy over the next 3 to 5 years." Wine Spectator Top 100 of 2019"  Wine Enthusiast Top 100 of 2018" "  Wine


In [20]:
descriptions_df = pd.DataFrame.from_dict(generated_descriptions, orient='index')
descriptions_df.columns = ['gpt2_description']
descriptions_df.to_csv('drive/MyDrive/data/wine/gpt2_descriptions.csv')

In [18]:
descriptions_df

,gpt2_description
Piul Bothen Cabernet Sauvignon 2014,[prompt] Piul Bothen Cabernet Sauvignon 2014 [...
Seacuscini Resantzass 2015,[prompt] Seacuscini Resantzass 2015 [response]...
Alpanena Vriestioge 2016,[prompt] Alpanena Vriestioge 2016 [response] [...
Sanmedarbecer Edena Cabernet,[prompt] Sanmedarbecer Edena Cabernet [respons...
Sauvignon 2001,[prompt] Sauvignon 2001 [response] [category_1...
...,...
Molphard Hidrmonied Viollia Colmy Sauvignon Blanc 2014,[prompt] Molphard Hidrmonied Viollia Colmy Sau...
Fersany Reserve Syrah 2012,[prompt] Fersany Reserve Syrah 2012 [response]...
Sambarat Palleviemle Wisersed Sesselle Cabernet Sauvignon 2015,[prompt] Sambarat Palleviemle Wisersed Sessell...
Talbamrioren Sheta Syrah 2005,[prompt] Talbamrioren Sheta Syrah 2005 [respon...
